In [176]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
df = pd.read_csv('../merged_tennis_files/tennis_matches_ml_with_elo.csv')

In [178]:
df

,surface,best_of,round,minutes,player_a_name,player_a_age,player_a_rank,player_a_height,player_a_hand,player_b_name,...,player_b_rank,player_b_height,player_b_hand,player_a_win,rank_diff,age_diff,height_diff,player_a_elo,player_b_elo,elo_diff
0,CARPET,3,F,135.0,Roger Taylor,33.3,41.0,183.0,L,Sandy Mayer,...,55.0,178.0,R,1,-14.0,10.5,5.0,1500.000000,1500.000000,0.000000
1,CLAY,5,F,136.0,Miloslav Mecir,22.9,5.0,190.0,R,Ivan Lendl,...,1.0,188.0,R,0,4.0,-4.2,2.0,1500.000000,1500.000000,0.000000
2,HARD,3,R32,70.0,Daniel Vacek,19.7,607.0,190.0,R,Markus Zoecke,...,167.0,196.0,R,0,440.0,-2.9,-6.0,1500.000000,1500.000000,0.000000
3,HARD,3,R16,80.0,Gilad Bloom,23.8,66.0,173.0,L,Michael Stich,...,42.0,193.0,R,0,24.0,1.6,-20.0,1500.000000,1500.000000,0.000000
4,HARD,3,R16,95.0,Nicklas Kulti,19.6,51.0,190.0,R,Magnus Gustafsson,...,31.0,185.0,R,1,20.0,-4.3,5.0,1500.000000,1500.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93398,HARD,5,RR,102.0,Learner Tien,19.0,122.0,180.0,L,Arthur Fils,...,20.0,185.0,R,1,102.0,-1.5,-5.0,1540.609555,1783.810162,-243.200607
93399,HARD,5,SF,95.0,Learner Tien,19.0,122.0,180.0,L,Alex Michelsen,...,41.0,193.0,R,1,81.0,-1.3,-13.0,1566.279314,1722.925180,-156.645867
93400,HARD,5,SF,60.0,Luca Van Assche,20.5,128.0,178.0,R,Joao Fonseca,...,145.0,185.0,R,0,-17.0,2.2,-7.0,1541.681828,1609.647630,-67.965802
93401,HARD,5,RR,67.0,Learner Tien,19.0,122.0,180.0,L,Joao Fonseca,...,145.0,185.0,R,0,-23.0,0.7,-5.0,1589.041045,1622.557013,-33.515968


In [179]:
model_df = df[["player_a_name", "player_b_name", "rank_diff", "age_diff", "height_diff", "elo_diff", "player_a_win"]]

In [180]:
model_df[:3]

,player_a_name,player_b_name,rank_diff,age_diff,height_diff,elo_diff,player_a_win
0,Roger Taylor,Sandy Mayer,-14.0,10.5,5.0,0.0,1
1,Miloslav Mecir,Ivan Lendl,4.0,-4.2,2.0,0.0,0
2,Daniel Vacek,Markus Zoecke,440.0,-2.9,-6.0,0.0,0


In [181]:
player_le = LabelEncoder()

all_players = pd.concat([model_df['player_a_name'], model_df['player_b_name']]).unique()

In [182]:
all_players

array(['Roger Taylor', 'Miloslav Mecir', 'Daniel Vacek', ...,
       'Kristjan Tamm', 'Andres Andrade', 'Branko Djuric'],
      shape=(1933,), dtype=object)

In [183]:
player_le.fit(all_players)

LabelEncoder()

In [184]:
model_df['player_a_id'] = player_le.transform(model_df['player_a_name'])
model_df['player_b_id'] = player_le.transform(model_df['player_b_name'])

/var/folders/pt/qgpr2mz91032lpscj9y1bjxh0000gn/T/ipykernel_57698/3466935981.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['player_a_id'] = player_le.transform(model_df['player_a_name'])
/var/folders/pt/qgpr2mz91032lpscj9y1bjxh0000gn/T/ipykernel_57698/3466935981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['player_b_id'] = player_le.transform(model_df['player_b_name'])


In [185]:
model_df

,player_a_name,player_b_name,rank_diff,age_diff,height_diff,elo_diff,player_a_win,player_a_id,player_b_id
0,Roger Taylor,Sandy Mayer,-14.0,10.5,5.0,0.000000,1,1606,1645
1,Miloslav Mecir,Ivan Lendl,4.0,-4.2,2.0,0.000000,0,1330,778
2,Daniel Vacek,Markus Zoecke,440.0,-2.9,-6.0,0.000000,0,390,1203
3,Gilad Bloom,Michael Stich,24.0,1.6,-20.0,0.000000,0,663,1296
4,Nicklas Kulti,Magnus Gustafsson,20.0,-4.3,5.0,0.000000,1,1361,1125
...,...,...,...,...,...,...,...,...,...
93398,Learner Tien,Arthur Fils,102.0,-1.5,-5.0,-243.200607,1,1069,187
93399,Learner Tien,Alex Michelsen,81.0,-1.3,-13.0,-156.645867,1,1069,67
93400,Luca Van Assche,Joao Fonseca,-17.0,2.2,-7.0,-67.965802,0,1096,899
93401,Learner Tien,Joao Fonseca,-23.0,0.7,-5.0,-33.515968,0,1069,899


In [186]:
FEATURES_NUMERIC = ["elo_diff", "height_diff", "age_diff", "rank_diff"]
TARGET = "player_a_win"


In [187]:
scaler = StandardScaler()
model_df[FEATURES_NUMERIC] = scaler.fit_transform(
    model_df[FEATURES_NUMERIC]
)

/var/folders/pt/qgpr2mz91032lpscj9y1bjxh0000gn/T/ipykernel_57698/2240220300.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[FEATURES_NUMERIC] = scaler.fit_transform(


In [188]:
train_df, test_df = train_test_split(
    model_df,
    test_size=0.2,
    shuffle=False
)

In [189]:
class TennisDataset(Dataset):
    def __init__(self, df):
        self.a_id = torch.tensor(df["player_a_id"].values, dtype=torch.long)
        self.b_id = torch.tensor(df["player_b_id"].values, dtype=torch.long)
        self.x_num = torch.tensor(
            df[FEATURES_NUMERIC].values,
            dtype=torch.float32
        )
        self.y = torch.tensor(df[TARGET].values, dtype=torch.float32)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return (
            self.a_id[idx],
            self.b_id[idx],
            self.x_num[idx],
            self.y[idx]
        )

In [190]:
BATCH_SIZE = 64

train_loader = DataLoader(
    TennisDataset(train_df),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    TennisDataset(test_df),
    batch_size=256,
    shuffle=False
)


In [191]:
class TennisMatchNN(nn.Module):
    def __init__(
        self,
        num_players,
        embedding_dim=32,
        num_numeric_features=4,
        dropout=0.3
    ):
        super().__init__()

        self.embedding = nn.Embedding(num_players, embedding_dim)

        self.numeric_fc = nn.Linear(num_numeric_features, 32)

        self.fc1 = nn.Linear(embedding_dim + 32, 128)
        self.bn1 = nn.BatchNorm1d(128)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)

        self.out = nn.Linear(64, 1)
        self.dropout = nn.Dropout(dropout)

        nn.init.xavier_uniform_(self.embedding.weight)

    def forward(self, a_id, b_id, x_num):
        emb_a = self.embedding(a_id)
        emb_b = self.embedding(b_id)

        emb_diff = emb_a - emb_b
        num_x = F.relu(self.numeric_fc(x_num))

        x = torch.cat([emb_diff, num_x], dim=1)

        x = self.dropout(F.relu(self.bn1(self.fc1(x))))
        x = self.dropout(F.relu(self.bn2(self.fc2(x))))

        return torch.sigmoid(self.out(x)).squeeze(1)


In [192]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_players = int(
    max(
        model_df["player_a_id"].max(),
        model_df["player_b_id"].max()
    ) + 1
)

model = TennisMatchNN(num_players).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4
)

In [193]:
EPOCHS = 50

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for a_id, b_id, x_num, y in train_loader:
        a_id = a_id.to(device)
        b_id = b_id.to(device)
        x_num = x_num.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        preds = model(a_id, b_id, x_num)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for a_id, b_id, x_num, y in test_loader:
            a_id = a_id.to(device)
            b_id = b_id.to(device)
            x_num = x_num.to(device)

            preds = model(a_id, b_id, x_num)

            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(y.numpy())

    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)

    accuracy = accuracy_score(all_targets, all_preds > 0.5)
    auc = roc_auc_score(all_targets, all_preds)

    #print(f"✅ Test Accuracy: {accuracy:.4f}")
    #print(f"✅ Test AUC: {auc:.4f}")

    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {train_loss/len(train_loader):.4f} | Test Accuracy: {accuracy:.4f} | Test AUC: {auc:.4f}")

Epoch 1/50 | Loss: 0.6181 | Test Accuracy: 0.6496 | Test AUC: 0.7115
Epoch 2/50 | Loss: 0.6116 | Test Accuracy: 0.6500 | Test AUC: 0.7105
Epoch 3/50 | Loss: 0.6067 | Test Accuracy: 0.6419 | Test AUC: 0.7013
Epoch 4/50 | Loss: 0.6012 | Test Accuracy: 0.6342 | Test AUC: 0.6875
Epoch 5/50 | Loss: 0.5958 | Test Accuracy: 0.6250 | Test AUC: 0.6781
Epoch 6/50 | Loss: 0.5920 | Test Accuracy: 0.6190 | Test AUC: 0.6688
Epoch 7/50 | Loss: 0.5876 | Test Accuracy: 0.6165 | Test AUC: 0.6601
Epoch 8/50 | Loss: 0.5847 | Test Accuracy: 0.6186 | Test AUC: 0.6647
Epoch 9/50 | Loss: 0.5807 | Test Accuracy: 0.6109 | Test AUC: 0.6530
Epoch 10/50 | Loss: 0.5765 | Test Accuracy: 0.6090 | Test AUC: 0.6485
Epoch 11/50 | Loss: 0.5721 | Test Accuracy: 0.6130 | Test AUC: 0.6508
Epoch 12/50 | Loss: 0.5696 | Test Accuracy: 0.6067 | Test AUC: 0.6418
Epoch 13/50 | Loss: 0.5651 | Test Accuracy: 0.6119 | Test AUC: 0.6492
Epoch 14/50 | Loss: 0.5613 | Test Accuracy: 0.6051 | Test AUC: 0.6348
Epoch 15/50 | Loss: 0.5584 | 